In [6]:
import pandas as pd
import requests


In [144]:
import warnings 
pd.options.mode.chained_assignment = None # se desactiva advertencias de pandas por edición_sobrescritura

In [3]:
df=pd.read_parquet("fhv_tripdata_2023-01.parquet")

In [10]:
# API

url = "https://data.cityofnewyork.us/resource/c3uy-2p5r.json"

# Parámetros de paginación
limit = 1000  # Número máximo de resultados por página
offset = 0    # Número de resultados a omitir antes de comenzar a obtener resultados
results = []  # Lista para almacenar los resultados

while True:
    # Construir la URL con los parámetros de paginación
    query_url = f'{url}?$limit={limit}&$offset={offset}'
    
    # Leer los datos de la URL en un DataFrame
    df = pd.read_json(query_url)
    
    # Agregar los resultados al DataFrame
    results.append(df)
    
    # Verificar si hay más resultados
    if len(df) < limit:
        break  # Si no hay más resultados, salir del ciclo
    
    offset += limit  # Incrementar el offset para la próxima página

# Concatenar todos los DataFrames en uno solo
df_final = pd.concat(results, ignore_index=True)

In [145]:
df_final

,unique_id,indicator_id,name,measure,measure_info,geo_type_name,geo_join_id,geo_place_name,time_period,start_date,data_value
0,216498,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),Summer 2013,2013-06-01T00:00:00.000,34.64
1,216499,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),Summer 2014,2014-06-01T00:00:00.000,33.22
2,219969,386,Ozone (O3),Mean,ppb,Borough,1,Bronx,Summer 2013,2013-06-01T00:00:00.000,31.25
3,219970,386,Ozone (O3),Mean,ppb,Borough,1,Bronx,Summer 2014,2014-06-01T00:00:00.000,31.15
4,164876,383,Sulfur Dioxide (SO2),Mean,ppb,CD,211,Morris Park and Bronxdale (CD11),Winter 2008-09,2008-12-01T00:00:00.000,5.89
...,...,...,...,...,...,...,...,...,...,...,...
16117,671118,386,Ozone (O3),Mean,ppb,CD,306,Park Slope and Carroll Gardens (CD6),Summer 2020,2020-06-01T00:00:00.000,28.70
16118,671119,386,Ozone (O3),Mean,ppb,CD,305,East New York and Starrett City (CD5),Summer 2020,2020-06-01T00:00:00.000,29.56
16119,671120,386,Ozone (O3),Mean,ppb,CD,304,Bushwick (CD4),Summer 2020,2020-06-01T00:00:00.000,29.65
16120,671121,386,Ozone (O3),Mean,ppb,CD,303,Bedford Stuyvesant (CD3),Summer 2020,2020-06-01T00:00:00.000,29.28


In [155]:
df=df_final

In [147]:
# Saco un dataset del trafico

trafic_1=df[df['name'] == 'Traffic Density- Annual Vehicle Miles Traveled for Cars']
trafic_2=df[df['name'] == 'Traffic Density- Annual Vehicle Miles Traveled']
trafic=pd.concat([trafic_1, trafic_2], axis=0, ignore_index=True)
trafic.head(5)

,unique_id,indicator_id,name,measure,measure_info,geo_type_name,geo_join_id,geo_place_name,time_period,start_date,data_value
0,151671,644,Traffic Density- Annual Vehicle Miles Traveled...,million miles,per km2,UHF42,101,Kingsbridge - Riverdale,2005,2005-01-01T00:00:00.000,22.6
1,151672,644,Traffic Density- Annual Vehicle Miles Traveled...,million miles,per km2,UHF42,102,Northeast Bronx,2005,2005-01-01T00:00:00.000,14.6
2,151673,644,Traffic Density- Annual Vehicle Miles Traveled...,million miles,per km2,UHF42,103,Fordham - Bronx Pk,2005,2005-01-01T00:00:00.000,22.8
3,151674,644,Traffic Density- Annual Vehicle Miles Traveled...,million miles,per km2,UHF42,104,Pelham - Throgs Neck,2005,2005-01-01T00:00:00.000,20.6
4,151675,644,Traffic Density- Annual Vehicle Miles Traveled...,million miles,per km2,UHF42,105,Crotona -Tremont,2005,2005-01-01T00:00:00.000,36.3


In [156]:
# Se elimina las filas de trafico del dataset

df = df[df['name'] != 'Traffic Density- Annual Vehicle Miles Traveled for Cars']
df = df[df['name'] != 'Traffic Density- Annual Vehicle Miles Traveled']
df = df[df['name'] != 'Traffic Density- Annual Vehicle Miles Traveled for Trucks']


# No interesa para el analisis, solo se valida las emisiones moviles de vehiculos
df = df[df['name'] != 'Boiler Emissions- Total SO2 Emissions']
df = df[df['name'] != 'Boiler Emissions- Total PM2.5 Emissions']
df = df[df['name'] != 'Boiler Emissions- Total NOx Emissions']

df=df.reset_index(drop=True)

La columna 'geo_type_name' determina las áreas de la ciudad. Para el análisis del EDA se usa **CD**. Equivalente a community districts.

In [166]:
print(f"Se selecciona CD con 5546 datos \n{df['geo_type_name'].value_counts()}")

CD_air=df[df['geo_type_name'] == 'CD']

CD_air=CD_air.reset_index(drop=True) #reset de indice

CD_air.head(3)

Se selecciona CD con 5546 datos 
UHF42       5712
CD          5546
UHF34       3128
Borough      680
Citywide     136
Name: geo_type_name, dtype: int64


,unique_id,indicator_id,name,measure,measure_info,geo_type_name,geo_join_id,geo_place_name,time_period,start_date,data_value
0,216498,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),Summer 2013,2013-06-01T00:00:00.000,34.64
1,216499,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),Summer 2014,2014-06-01T00:00:00.000,33.22
2,164876,383,Sulfur Dioxide (SO2),Mean,ppb,CD,211,Morris Park and Bronxdale (CD11),Winter 2008-09,2008-12-01T00:00:00.000,5.89


In [167]:
CD_air['name'].value_counts()

Nitrogen Dioxide (NO2)                                            2124
Fine Particulate Matter (PM2.5)                                   2124
Ozone (O3)                                                         708
Sulfur Dioxide (SO2)                                               472
Air Toxics Concentrations- Average Formaldehyde Concentrations      59
Air Toxics Concentrations- Average Benzene Concentrations           59
Name: name, dtype: int64

In [168]:
# Se renombra categorias 

# Avg Benzene 
CD_air['name'] = CD_air['name'].apply(lambda x: "Avg Benzene" if 'Average Benzene Concentrations' in x else  x)

# Avg Formaldehyde 
CD_air['name'] = CD_air['name'].apply(lambda x: "Avg Formaldehyde" if 'Formaldehyde Concentrations' in x else  x)

In [169]:
CD_air['name'].value_counts()

Nitrogen Dioxide (NO2)             2124
Fine Particulate Matter (PM2.5)    2124
Ozone (O3)                          708
Sulfur Dioxide (SO2)                472
Avg Formaldehyde                     59
Avg Benzene                          59
Name: name, dtype: int64

In [170]:
# se valida tipo de datos
CD_air.dtypes


unique_id           int64
indicator_id        int64
name               object
measure            object
measure_info       object
geo_type_name      object
geo_join_id         int64
geo_place_name     object
time_period        object
start_date         object
data_value        float64
dtype: object

In [171]:
# Separo str en la columna time_period me quedo solo con la estación del año
CD_air['time_period'] = CD_air['time_period'].str.split(' ').str[0]
CD_air.sample(3)

,unique_id,indicator_id,name,measure,measure_info,geo_type_name,geo_join_id,geo_place_name,time_period,start_date,data_value
4216,550076,375,Nitrogen Dioxide (NO2),Mean,ppb,CD,305,East New York and Starrett City (CD5),Annual,2017-01-01T00:00:00.000,17.92
1051,167673,375,Nitrogen Dioxide (NO2),Mean,ppb,CD,405,Ridgewood and Maspeth (CD5),Annual,2010-12-01T00:00:00.000,23.31
3309,337831,383,Sulfur Dioxide (SO2),Mean,ppb,CD,106,Stuyvesant Town and Turtle Bay (CD6),Winter,2014-12-01T00:00:00.000,1.20


In [178]:
# Community Districts New york

brooklyn = ["South Crown Heights and Lefferts Gardens (CD9)","Sunset Park (CD7)","Sheepshead Bay (CD15)",
"Park Slope and Carroll Gardens (CD6)", "Greenpoint and Williamsburg (CD1)", "Fort Greene and Brooklyn Heights (CD2)",
"Flatlands and Canarsie (CD18)","Flatbush and Midwood (CD14)","East New York and Starrett City (CD5)","East Flatbush (CD17)",
"Crown Heights and Prospect Heights (CD8)", "Coney Island (CD13)","Bushwick (CD4)","Brownsville (CD16)",
"Borough Park (CD12)","Bensonhurst (CD11)","Bedford Stuyvesant (CD3)", "Bay Ridge and Dyker Heights (CD10)"]

manhattan = ["Financial District (CD1)", "Greenwich Village and Soho (CD2)","Lower East Side and Chinatown (CD3)",
    "Clinton and Chelsea (CD4)","Midtown (CD5)","Stuyvesant Town and Turtle Bay (CD6)","Upper West Side (CD7)",
    "Upper East Side (CD8)","Morningside Heights and Hamilton Heights (CD9)","Central Harlem (CD10)","East Harlem (CD11)",
    "Washington Heights and Inwood (CD12)"]

queens = ["Long Island City and Astoria (CD1)","Woodside and Sunnyside (CD2)","Elmhurst and Corona (CD4)",
    "Ridgewood and Maspeth (CD5)","Rego Park and Forest Hills (CD6)","Flushing and Whitestone (CD7)",
    "Hillcrest and Fresh Meadows (CD8)","Kew Gardens and Woodhaven (CD9)","South Ozone Park and Howard Beach (CD10)",
    "Bayside and Little Neck (CD11)","Jamaica and Hollis (CD12)","Queens Village (CD13)", "Rockaway and Broad Channel (CD14)",
    "Jackson Heights (CD3)"]

state_island = ["St. George and Stapleton (CD1)","Tottenville and Great Kills (CD3)","South Beach and Willowbrook (CD2)"]

the_bronx = ["Morrisania and Crotona (CD3)", "Highbridge and Concourse (CD4)", "Hunts Point and Longwood (CD2)",
              "Morris Park and Bronxdale (CD11)", "Parkchester and Soundview (CD9)","Williamsbridge and Baychester (CD12)",
              "Belmont and East Tremont (CD6)", "Kingsbridge Heights and Bedford (CD7)", "Riverdale and Fieldston (CD8)",
              "Throgs Neck and Co-op City (CD10)","Mott Haven and Melrose (CD1)", "Fordham and University Heights (CD5)"]




In [172]:
CD_air['borough']='borough'



In [188]:
CD_air.head(2)

,unique_id,indicator_id,name,measure,measure_info,geo_type_name,geo_join_id,geo_place_name,time_period,start_date,data_value,borough,borout
0,216498,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),Summer,2013-06-01T00:00:00.000,34.64,borough,None
1,216499,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),Summer,2014-06-01T00:00:00.000,33.22,borough,None


In [197]:
# Definir una función para verificar si el valor está en la lista
def en_state_island(valor):
    return valor in state_island

# Crear la nueva columna y asignar el valor correspondiente

CD_air['borough'] = CD_air['geo_place_name'].apply(lambda x:'brooklyn' if x in brooklyn else 
                                                          ('manhattan' if x in manhattan else 
                                                          ('queens' if x in queens else 
                                                          ('state_island' if x in state_island else 
                                                          ('the_bronx' if  x in the_bronx else None)))))


In [198]:
#CD_air[CD_air['borout'] == 'state_island']
CD_air

,unique_id,indicator_id,name,measure,measure_info,geo_type_name,geo_join_id,geo_place_name,time_period,start_date,data_value,borough,borout
0,216498,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),Summer,2013-06-01T00:00:00.000,34.64,brooklyn,None
1,216499,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),Summer,2014-06-01T00:00:00.000,33.22,brooklyn,None
2,164876,383,Sulfur Dioxide (SO2),Mean,ppb,CD,211,Morris Park and Bronxdale (CD11),Winter,2008-12-01T00:00:00.000,5.89,the_bronx,None
3,164877,383,Sulfur Dioxide (SO2),Mean,ppb,CD,212,Williamsbridge and Baychester (CD12),Winter,2008-12-01T00:00:00.000,5.75,the_bronx,None
4,164878,383,Sulfur Dioxide (SO2),Mean,ppb,CD,301,Greenpoint and Williamsburg (CD1),Winter,2008-12-01T00:00:00.000,4.33,brooklyn,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5541,671118,386,Ozone (O3),Mean,ppb,CD,306,Park Slope and Carroll Gardens (CD6),Summer,2020-06-01T00:00:00.000,28.70,brooklyn,None
5542,671119,386,Ozone (O3),Mean,ppb,CD,305,East New York and Starrett City (CD5),Summer,2020-06-01T00:00:00.000,29.56,brooklyn,None
5543,671120,386,Ozone (O3),Mean,ppb,CD,304,Bushwick (CD4),Summer,2020-06-01T00:00:00.000,29.65,brooklyn,None
5544,671121,386,Ozone (O3),Mean,ppb,CD,303,Bedford Stuyvesant (CD3),Summer,2020-06-01T00:00:00.000,29.28,brooklyn,None


In [190]:
en_state_island('St. George and Stapleton (CD1)')

True

In [ ]:
CD_air[CD_air['borough'] == 'state_island']

In [41]:
a['index'].tolist()

['Long Island City - Astoria',
 'West Queens',
 'East New York',
 'Bedford Stuyvesant - Crown Heights',
 'Downtown - Heights - Slope',
 'Greenpoint',
 'East Flatbush - Flatbush',
 'Canarsie - Flatlands',
 'Bensonhurst - Bay Ridge',
 'Pelham - Throgs Neck',
 'Fordham - Bronx Pk',
 'Northeast Bronx',
 'Kingsbridge - Riverdale',
 'Flushing - Clearview',
 'Borough Park',
 'Coney Island - Sheepshead Bay',
 'Ridgewood - Forest Hills',
 'Williamsburg - Bushwick',
 'Southwest Queens',
 'Jamaica',
 'Southeast Queens',
 'Rockaways',
 'Washington Heights',
 'Central Harlem - Morningside Heights',
 'East Harlem',
 'Upper West Side',
 'Sunset Park',
 'South Beach - Tottenville',
 'Union Square - Lower East Side',
 'Lower Manhattan',
 'Gramercy Park - Murray Hill',
 'Bayside - Little Neck',
 'Fresh Meadows',
 'Port Richmond',
 'Stapleton - St. George',
 'Willowbrook',
 'New York City',
 'Greenwich Village - SoHo',
 'Chelsea - Clinton',
 'Hunts Point - Mott Haven',
 'High Bridge - Morrisania',
 'Crot

In [14]:
# Se exporta dataset

df_final.to_parquet("df_final.parquet",index=False)     #dataframe 1